In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
def datetime_to_str(dtime):
    return '{0}-{1}-{2}'.format(str(dtime.month).zfill(2), str(dtime.day).zfill(2), str(dtime.year).zfill(4))
# datetime_to_str(datetime.now())

def datetime_to_str2(dtime):
    return '{0}{1}{2}'.format(str(dtime.year).zfill(4), str(dtime.month).zfill(2), str(dtime.day).zfill(2))
# datetime_to_str2(datetime.now())

In [3]:
ndays=31
now = datetime.now() - timedelta(days=1)

cols = [
    'Total Distributed',
    'Total Administered',
    'Distributed per 100K',
    'Administered per 100K',
    'People with 1+ Doses',
    'People with 1+ Doses per 100K',
    'People with 2 Doses',
    'People with 2 Doses Per 100K',
]

def get_vaccination_data(dtime, col):
    exclude = [
        'Diamond Princess',
        'Grand Princess',
        'American Samoa',
        'Virgin Islands',
        'Northern Mariana Islands'
    ]
    file = 'covid19_vaccinations_in_the_united_states-{0}.csv'.format(datetime_to_str2(dtime))
    df = pd.read_csv('vaccinations/{0}'.format(file),
                     usecols=[ 'State/Territory/Federal Entity', col]).fillna(0)
#     if dtime >= datetime(2021, 2, 5):
#         df.rename(columns={
#                     'State/Territory/Federal Entity': 'State', '{0}'.format(col): datetime_to_str(dtime),
#                     'Total Delivered': 'Total Distributed'
#                   },
#                   inplace=True)
#     else:
    df.rename(columns={ 'State/Territory/Federal Entity': 'State', '{0}'.format(col): datetime_to_str(dtime) }, inplace=True)
    df.set_index(['State'], inplace=True)
    return df

def get_vaccination_timeseries(dtime, col, ndays=5):
    df = get_vaccination_data(dtime, col)
    for days in range(1, ndays+1):
        df_2 = get_vaccination_data(dtime - timedelta(days=days), col)
        df = pd.concat([df_2, df], axis=1, join='inner')
    return df

In [4]:
df = get_vaccination_data(now, col='Total Doses Delivered')
df

,03-31-2021
State,
Alaska,615825
Alabama,2839540
Arkansas,1814280
American Samoa,43810
Arizona,4365955
...,...
Vermont,432900
Washington,4571720
Wisconsin,3399265


In [5]:
df.sum()

03-31-2021    209503735
dtype: int64

In [6]:
df = get_vaccination_data(now, col='Total Doses Administered by State where Administered')
df

,03-31-2021
State,
Alaska,411529
Alabama,1771979
Arkansas,1209965
American Samoa,31167
Arizona,3512579
...,...
Vermont,333676
Washington,3659886
Wisconsin,2969294


In [7]:
df.sum()

03-31-2021    160018257
dtype: int64

In [8]:
df = get_vaccination_timeseries(now, 'Total Doses Delivered', ndays)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,342535,372725,399225,409125,421375,426715,426715,426715,430685,445125,...,529765,540815,549315,568125,570465,570465,570465,572905,601225,615825
Alabama,1402280,1461320,1469100,1530470,1556460,1628350,1628350,1628350,1689050,1775660,...,2275030,2403720,2459980,2497410,2535880,2537050,2537050,2722700,2796630,2839540
Arkansas,892660,963040,964840,990640,1064920,1088150,1088150,1088150,1158630,1162330,...,1535620,1537720,1576060,1608210,1662850,1662850,1662850,1744560,1763540,1814280
American Samoa,29450,29450,29450,38810,43210,43210,43210,43210,43210,43210,...,43210,43210,43810,43810,43810,43810,43810,43810,43810,43810
Arizona,2148015,2244885,2392275,2453985,2598435,2623735,2623735,2623735,2638855,2887485,...,3584605,3734465,3797605,3940655,3970345,3970345,3970345,4153635,4304045,4365955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,206910,227350,230850,246000,253300,261100,261100,261100,272530,281530,...,353420,355290,374580,379490,393330,393330,393330,409540,417840,432900
Washington,2245010,2443300,2517210,2572750,2628620,2657880,2657880,2657880,2942240,2989200,...,3897910,3927360,3974140,4040730,4114320,4114320,4114320,4325160,4495070,4571720
Wisconsin,1588665,1814215,1863155,1876225,1895165,1924765,1924865,1924865,2154685,2169885,...,2875615,2884825,2923795,2938835,2996415,2996415,2996415,3348455,3355325,3399265


In [9]:
df = get_vaccination_timeseries(now, 'Total Doses Administered by State where Administered', ndays)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,268406,269941,273350,276891,281467,288625,295046,298144,302687,307769,...,376555,379296,384503,389775,395650,401502,403284,404339,407233,411529
Alabama,948393,972161,1000031,1032171,1064578,1099697,1110132,1116676,1140984,1173820,...,1509445,1542416,1579091,1611404,1645858,1659102,1666585,1697641,1728736,1771979
Arkansas,661572,670815,687433,704368,723756,744139,765260,773357,782750,799184,...,1012415,1032499,1056307,1086176,1114449,1143893,1153739,1159108,1183295,1209965
American Samoa,20338,21079,21079,21625,21625,21625,21625,21625,21625,23696,...,27263,27837,27864,28503,28527,28527,28527,29079,31167,31167
Arizona,1849259,1889742,1930999,1979929,2038623,2043739,2162602,2215219,2245854,2293559,...,2970577,3010048,3106598,3169061,3239538,3308671,3363062,3399399,3451567,3512579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,168050,171557,173179,175767,179580,184327,191529,198741,203900,207110,...,283052,286384,290680,296682,303242,310003,316145,323388,328451,333676
Washington,1800030,1841160,1888198,1942877,2001032,2071467,2143916,2213284,2255182,2303420,...,3068976,3108315,3170650,3251437,3330877,3419568,3501641,3537203,3586736,3659886
Wisconsin,1500923,1521200,1553270,1596290,1656753,1721352,1770069,1804918,1812456,1839822,...,2480311,2516425,2576727,2654772,2736745,2808953,2853949,2870834,2905410,2969294


In [10]:
df = get_vaccination_timeseries(now, 'People with 1+ Doses by State of Residence', ndays)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,168069,168646,170586,172414,174588,178237,180948,182230,184906,188281,...,231342,233037,236062,239599,242733,245355,246515,247097,248711,250378
Alabama,653309,663187,674810,688791,704173,721109,727681,732967,747268,768502,...,994253,1019858,1048838,1072623,1098597,1108253,1113924,1134739,1155139,1184284
Arkansas,425567,431866,442371,452615,464641,477127,489149,494761,501653,513591,...,680549,694124,710494,730820,747412,763783,769106,772457,788277,803744
American Samoa,12355,13028,13029,13467,13468,13468,13469,13469,13471,15085,...,17114,17371,17385,17548,17567,17570,17575,17718,17856,17856
Arizona,1244879,1262868,1277294,1298827,1325003,1328642,1388900,1416598,1435152,1464494,...,1851810,1877865,1940953,1983569,2030110,2072778,2105116,2125696,2157694,2194105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,107238,109213,110313,111890,114893,118292,122891,126974,130466,132811,...,183362,185840,188433,192146,196126,199645,203266,206956,209145,212105
Washington,1166618,1186560,1210794,1243596,1276221,1321095,1367009,1411252,1437268,1469986,...,1982908,2008970,2046826,2097456,2142877,2188808,2232951,2250118,2272698,2309959
Wisconsin,971107,984053,1001656,1024560,1059657,1098730,1128307,1150440,1155268,1171932,...,1584076,1605873,1647536,1701809,1752800,1794367,1819527,1829412,1850625,1890610


In [11]:
df = get_vaccination_timeseries(now, 'People with 2 Doses by State of Residence', ndays)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,99916,100902,102382,104129,106558,110053,113787,115633,117341,119067,...,144278,145460,147600,149303,152015,155198,155843,156322,157589,159632
Alabama,316073,330046,346841,365456,383078,402061,406723,408318,418443,430554,...,550476,558815,567276,576529,586065,590627,593437,603928,615577,630815
Arkansas,233685,236706,242999,249956,257367,265226,274376,277102,279415,284169,...,331747,338390,346045,354603,366228,378793,383151,385265,393829,405237
American Samoa,8080,8142,8142,8252,8252,8252,8252,8253,8253,8705,...,10237,10549,10563,11040,11043,11043,11045,11456,13402,13402
Arizona,555881,575562,601274,627446,658220,662235,715023,737536,748413,764433,...,1032516,1045360,1075612,1094137,1116392,1141583,1162369,1177421,1196441,1219181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,56934,58448,58977,59938,60708,61961,64474,67478,69135,69904,...,93242,94023,95560,97585,100051,103234,105663,109128,111966,114213
Washington,621171,642362,664542,686099,711273,736314,762728,787532,802793,817686,...,1065852,1078709,1102949,1132513,1166105,1208425,1245782,1263593,1290291,1325836
Wisconsin,517730,525277,539106,558736,583220,607958,627022,639217,641825,652234,...,872159,885784,903562,926731,956643,986762,1005175,1011749,1024801,1047289


In [12]:
df = get_vaccination_timeseries(now, 'Total Doses Delivered', ndays).diff(axis=1).fillna(0)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,0.0,30190.0,26500.0,9900.0,12250.0,5340.0,0.0,0.0,3970.0,14440.0,...,9080.0,11050.0,8500.0,18810.0,2340.0,0.0,0.0,2440.0,28320.0,14600.0
Alabama,0.0,59040.0,7780.0,61370.0,25990.0,71890.0,0.0,0.0,60700.0,86610.0,...,97150.0,128690.0,56260.0,37430.0,38470.0,1170.0,0.0,185650.0,73930.0,42910.0
Arkansas,0.0,70380.0,1800.0,25800.0,74280.0,23230.0,0.0,0.0,70480.0,3700.0,...,68670.0,2100.0,38340.0,32150.0,54640.0,0.0,0.0,81710.0,18980.0,50740.0
American Samoa,0.0,0.0,0.0,9360.0,4400.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arizona,0.0,96870.0,147390.0,61710.0,144450.0,25300.0,0.0,0.0,15120.0,248630.0,...,123360.0,149860.0,63140.0,143050.0,29690.0,0.0,0.0,183290.0,150410.0,61910.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,0.0,20440.0,3500.0,15150.0,7300.0,7800.0,0.0,0.0,11430.0,9000.0,...,10460.0,1870.0,19290.0,4910.0,13840.0,0.0,0.0,16210.0,8300.0,15060.0
Washington,0.0,198290.0,73910.0,55540.0,55870.0,29260.0,0.0,0.0,284360.0,46960.0,...,311030.0,29450.0,46780.0,66590.0,73590.0,0.0,0.0,210840.0,169910.0,76650.0
Wisconsin,0.0,225550.0,48940.0,13070.0,18940.0,29600.0,100.0,0.0,229820.0,15200.0,...,260650.0,9210.0,38970.0,15040.0,57580.0,0.0,0.0,352040.0,6870.0,43940.0


In [13]:
df = get_vaccination_timeseries(now, 'Total Doses Administered by State where Administered', ndays).diff(axis=1).fillna(0)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,0.0,1535.0,3409.0,3541.0,4576.0,7158.0,6421.0,3098.0,4543.0,5082.0,...,965.0,2741.0,5207.0,5272.0,5875.0,5852.0,1782.0,1055.0,2894.0,4296.0
Alabama,0.0,23768.0,27870.0,32140.0,32407.0,35119.0,10435.0,6544.0,24308.0,32836.0,...,30174.0,32971.0,36675.0,32313.0,34454.0,13244.0,7483.0,31056.0,31095.0,43243.0
Arkansas,0.0,9243.0,16618.0,16935.0,19388.0,20383.0,21121.0,8097.0,9393.0,16434.0,...,3840.0,20084.0,23808.0,29869.0,28273.0,29444.0,9846.0,5369.0,24187.0,26670.0
American Samoa,0.0,741.0,0.0,546.0,0.0,0.0,0.0,0.0,0.0,2071.0,...,248.0,574.0,27.0,639.0,24.0,0.0,0.0,552.0,2088.0,0.0
Arizona,0.0,40483.0,41257.0,48930.0,58694.0,5116.0,118863.0,52617.0,30635.0,47705.0,...,27670.0,39471.0,96550.0,62463.0,70477.0,69133.0,54391.0,36337.0,52168.0,61012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,0.0,3507.0,1622.0,2588.0,3813.0,4747.0,7202.0,7212.0,5159.0,3210.0,...,5632.0,3332.0,4296.0,6002.0,6560.0,6761.0,6142.0,7243.0,5063.0,5225.0
Washington,0.0,41130.0,47038.0,54679.0,58155.0,70435.0,72449.0,69368.0,41898.0,48238.0,...,23379.0,39339.0,62335.0,80787.0,79440.0,88691.0,82073.0,35562.0,49533.0,73150.0
Wisconsin,0.0,20277.0,32070.0,43020.0,60463.0,64599.0,48717.0,34849.0,7538.0,27366.0,...,18230.0,36114.0,60302.0,78045.0,81973.0,72208.0,44996.0,16885.0,34576.0,63884.0


In [14]:
df = get_vaccination_timeseries(now, 'People with 1+ Doses by State of Residence', ndays).diff(axis=1).fillna(0)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,0.0,577.0,1940.0,1828.0,2174.0,3649.0,2711.0,1282.0,2676.0,3375.0,...,728.0,1695.0,3025.0,3537.0,3134.0,2622.0,1160.0,582.0,1614.0,1667.0
Alabama,0.0,9878.0,11623.0,13981.0,15382.0,16936.0,6572.0,5286.0,14301.0,21234.0,...,21267.0,25605.0,28980.0,23785.0,25974.0,9656.0,5671.0,20815.0,20400.0,29145.0
Arkansas,0.0,6299.0,10505.0,10244.0,12026.0,12486.0,12022.0,5612.0,6892.0,11938.0,...,2743.0,13575.0,16370.0,20326.0,16592.0,16371.0,5323.0,3351.0,15820.0,15467.0
American Samoa,0.0,673.0,1.0,438.0,1.0,0.0,1.0,0.0,2.0,1614.0,...,198.0,257.0,14.0,163.0,19.0,3.0,5.0,143.0,138.0,0.0
Arizona,0.0,17989.0,14426.0,21533.0,26176.0,3639.0,60258.0,27698.0,18554.0,29342.0,...,16771.0,26055.0,63088.0,42616.0,46541.0,42668.0,32338.0,20580.0,31998.0,36411.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,0.0,1975.0,1100.0,1577.0,3003.0,3399.0,4599.0,4083.0,3492.0,2345.0,...,3637.0,2478.0,2593.0,3713.0,3980.0,3519.0,3621.0,3690.0,2189.0,2960.0
Washington,0.0,19942.0,24234.0,32802.0,32625.0,44874.0,45914.0,44243.0,26016.0,32718.0,...,14600.0,26062.0,37856.0,50630.0,45421.0,45931.0,44143.0,17167.0,22580.0,37261.0
Wisconsin,0.0,12946.0,17603.0,22904.0,35097.0,39073.0,29577.0,22133.0,4828.0,16664.0,...,13478.0,21797.0,41663.0,54273.0,50991.0,41567.0,25160.0,9885.0,21213.0,39985.0


In [15]:
df = get_vaccination_timeseries(now, 'People with 2 Doses by State of Residence', ndays).diff(axis=1).fillna(0)
df

,02-28-2021,03-01-2021,03-02-2021,03-03-2021,03-04-2021,03-05-2021,03-06-2021,03-07-2021,03-08-2021,03-09-2021,...,03-22-2021,03-23-2021,03-24-2021,03-25-2021,03-26-2021,03-27-2021,03-28-2021,03-29-2021,03-30-2021,03-31-2021
State,,,,,,,,,,,,,,,,,,,,,
Alaska,0.0,986.0,1480.0,1747.0,2429.0,3495.0,3734.0,1846.0,1708.0,1726.0,...,237.0,1182.0,2140.0,1703.0,2712.0,3183.0,645.0,479.0,1267.0,2043.0
Alabama,0.0,13973.0,16795.0,18615.0,17622.0,18983.0,4662.0,1595.0,10125.0,12111.0,...,9071.0,8339.0,8461.0,9253.0,9536.0,4562.0,2810.0,10491.0,11649.0,15238.0
Arkansas,0.0,3021.0,6293.0,6957.0,7411.0,7859.0,9150.0,2726.0,2313.0,4754.0,...,1187.0,6643.0,7655.0,8558.0,11625.0,12565.0,4358.0,2114.0,8564.0,11408.0
American Samoa,0.0,62.0,0.0,110.0,0.0,0.0,0.0,1.0,0.0,452.0,...,44.0,312.0,14.0,477.0,3.0,0.0,2.0,411.0,1946.0,0.0
Arizona,0.0,19681.0,25712.0,26172.0,30774.0,4015.0,52788.0,22513.0,10877.0,16020.0,...,10219.0,12844.0,30252.0,18525.0,22255.0,25191.0,20786.0,15052.0,19020.0,22740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermont,0.0,1514.0,529.0,961.0,770.0,1253.0,2513.0,3004.0,1657.0,769.0,...,1938.0,781.0,1537.0,2025.0,2466.0,3183.0,2429.0,3465.0,2838.0,2247.0
Washington,0.0,21191.0,22180.0,21557.0,25174.0,25041.0,26414.0,24804.0,15261.0,14893.0,...,8849.0,12857.0,24240.0,29564.0,33592.0,42320.0,37357.0,17811.0,26698.0,35545.0
Wisconsin,0.0,7547.0,13829.0,19630.0,24484.0,24738.0,19064.0,12195.0,2608.0,10409.0,...,4417.0,13625.0,17778.0,23169.0,29912.0,30119.0,18413.0,6574.0,13052.0,22488.0


In [16]:
df = get_vaccination_timeseries(now, 'Total Doses Delivered', ndays).sum(axis=0).diff().fillna(0)
df

02-28-2021          0.0
03-01-2021    6154410.0
03-02-2021    4864640.0
03-03-2021    3514360.0
03-04-2021    4298605.0
03-05-2021    2258420.0
03-06-2021       8000.0
03-07-2021      15210.0
03-08-2021    7066720.0
03-09-2021    4869980.0
03-10-2021    3387775.0
03-11-2021    2323220.0
03-12-2021    2512240.0
03-13-2021       1170.0
03-14-2021          0.0
03-15-2021    7268310.0
03-16-2021    4966310.0
03-17-2021    3715060.0
03-18-2021    3188190.0
03-19-2021    2538990.0
03-20-2021          0.0
03-21-2021          0.0
03-22-2021    7811640.0
03-23-2021    5077530.0
03-24-2021    4532450.0
03-25-2021    4086360.0
03-26-2021    3146720.0
03-27-2021       2340.0
03-28-2021        100.0
03-29-2021    9068670.0
03-30-2021    6342360.0
03-31-2021    5123890.0
dtype: float64

In [17]:
df = get_vaccination_timeseries(now, 'Total Doses Administered by State where Administered', ndays).sum(axis=0).diff().fillna(0)
df

02-28-2021          0.0
03-01-2021    1757841.0
03-02-2021    1984130.0
03-03-2021    2124031.0
03-04-2021    2535228.0
03-05-2021    3012371.0
03-06-2021    2540886.0
03-07-2021    1807936.0
03-08-2021    1695847.0
03-09-2021    2120647.0
03-10-2021    2591099.0
03-11-2021    3040003.0
03-12-2021    4770277.0
03-13-2021    1369810.0
03-14-2021    2147304.0
03-15-2021    1685454.0
03-16-2021    2384089.0
03-17-2021    2796214.0
03-18-2021    2684496.0
03-19-2021    3225218.0
03-20-2021    3149984.0
03-21-2021    2094488.0
03-22-2021    1721831.0
03-23-2021    2327285.0
03-24-2021    2911025.0
03-25-2021    3509790.0
03-26-2021    3605017.0
03-27-2021    3401378.0
03-28-2021    2415815.0
03-29-2021    1800130.0
03-30-2021    2758318.0
03-31-2021    3460313.0
dtype: float64

In [18]:
df = get_vaccination_timeseries(now, 'Total Doses Delivered', ndays).diff(axis=1).fillna(0)
df.loc['Connecticut']

02-28-2021         0.0
03-01-2021     83440.0
03-02-2021     27290.0
03-03-2021     36370.0
03-04-2021     75450.0
03-05-2021     13520.0
03-06-2021         0.0
03-07-2021         0.0
03-08-2021     91350.0
03-09-2021     32530.0
03-10-2021     46440.0
03-11-2021     37740.0
03-12-2021     16460.0
03-13-2021         0.0
03-14-2021         0.0
03-15-2021     90130.0
03-16-2021     25340.0
03-17-2021     45970.0
03-18-2021     64170.0
03-19-2021     14700.0
03-20-2021         0.0
03-21-2021         0.0
03-22-2021     76460.0
03-23-2021     94950.0
03-24-2021     26350.0
03-25-2021     95920.0
03-26-2021     18200.0
03-27-2021         0.0
03-28-2021         0.0
03-29-2021    111140.0
03-30-2021    104040.0
03-31-2021     58140.0
Name: Connecticut, dtype: float64

In [19]:
df = get_vaccination_timeseries(now, 'Total Doses Administered by State where Administered', ndays).diff(axis=1).fillna(0)
df.loc['Connecticut']

02-28-2021        0.0
03-01-2021     8849.0
03-02-2021    21221.0
03-03-2021    25938.0
03-04-2021    34204.0
03-05-2021    35597.0
03-06-2021    38503.0
03-07-2021    33248.0
03-08-2021    16822.0
03-09-2021    26894.0
03-10-2021    31721.0
03-11-2021    41048.0
03-12-2021    66175.0
03-13-2021        7.0
03-14-2021    30907.0
03-15-2021    17895.0
03-16-2021    23253.0
03-17-2021    31358.0
03-18-2021    36536.0
03-19-2021    35003.0
03-20-2021    36606.0
03-21-2021    35248.0
03-22-2021    18567.0
03-23-2021    24393.0
03-24-2021    35862.0
03-25-2021    36885.0
03-26-2021    39809.0
03-27-2021    40435.0
03-28-2021    37314.0
03-29-2021    33002.0
03-30-2021    29654.0
03-31-2021    38640.0
Name: Connecticut, dtype: float64

In [20]:
df = get_vaccination_timeseries(now, 'People with 1+ Doses by State of Residence', ndays).diff(axis=1).fillna(0)
df.loc['Connecticut']

02-28-2021        0.0
03-01-2021     7196.0
03-02-2021    17277.0
03-03-2021    22854.0
03-04-2021    30337.0
03-05-2021    32067.0
03-06-2021    33942.0
03-07-2021    27640.0
03-08-2021    13888.0
03-09-2021    23019.0
03-10-2021    27980.0
03-11-2021    35809.0
03-12-2021   -51157.0
03-13-2021      140.0
03-14-2021    19142.0
03-15-2021    12197.0
03-16-2021    12449.0
03-17-2021    16723.0
03-18-2021    20733.0
03-19-2021    19828.0
03-20-2021    23066.0
03-21-2021    21865.0
03-22-2021    14113.0
03-23-2021    15069.0
03-24-2021    22980.0
03-25-2021    23707.0
03-26-2021    22921.0
03-27-2021    24156.0
03-28-2021    23164.0
03-29-2021    18904.0
03-30-2021    18119.0
03-31-2021    21755.0
Name: Connecticut, dtype: float64

In [21]:
df = get_vaccination_timeseries(now, 'People with 2 Doses by State of Residence', ndays).diff(axis=1).fillna(0)
df.loc['Connecticut']

02-28-2021         0.0
03-01-2021      1611.0
03-02-2021      3819.0
03-03-2021      3003.0
03-04-2021      3517.0
03-05-2021      3459.0
03-06-2021      4438.0
03-07-2021      5665.0
03-08-2021      2914.0
03-09-2021      4001.0
03-10-2021      3879.0
03-11-2021      5214.0
03-12-2021    116769.0
03-13-2021        38.0
03-14-2021     11880.0
03-15-2021      5668.0
03-16-2021     11010.0
03-17-2021     14548.0
03-18-2021     15787.0
03-19-2021     14923.0
03-20-2021     13464.0
03-21-2021     13289.0
03-22-2021      4625.0
03-23-2021      9472.0
03-24-2021     12817.0
03-25-2021     13152.0
03-26-2021     17016.0
03-27-2021     16432.0
03-28-2021     14148.0
03-29-2021     13542.0
03-30-2021     11489.0
03-31-2021     16549.0
Name: Connecticut, dtype: float64